# Initial Setups

## Set MSCTD Configurations

In [1]:
!pip install --upgrade --no-cache-dir gdown
!pip3 install --upgrade gdown
!pip install -U --no-cache-dir gdown --pre
!pip install --upgrade --no-cache-dir gdown
# 
!git clone https://github.com/XL2248/MSCTD

!cp MSCTD/MSCTD_data/ende/english_*.txt .
!cp MSCTD/MSCTD_data/ende/image_index_*.txt .
!cp MSCTD/MSCTD_data/ende/sentiment*.txt .

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'MSCTD'...
remote: Enumerating objects: 1217, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1217 (delta 13), reused 7 (delta 3), pack-reused 1190
Receiving objects: 100% (1217/1217), 102.24 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (616/616), done.
Updating files: 100% (934/934), done.
Mounted at /content/drive


In [2]:
!pip install einops
!pip install pytorch_lightning
!pip install wandb
!pip install ml_collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 41.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.4 MB/s eta 0:00:00
  Created wheel for

## Import Libraries

In [3]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
from tqdm.notebook import tqdm
from time import localtime
import gc

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from IPython.display import clear_output 
from PIL import Image, ImageOps, ImageEnhance

import pytorch_lightning as pl
import torchvision.datasets as dset
import torchvision.transforms as T
from matplotlib import image as mpimg
from PIL import Image
from PIL import Image, ImageOps, ImageEnhance
from einops import parse_shape, rearrange
from opt_einsum import contract
import math
import functools
import pickle, marshal
from einops import rearrange
from numpy import asarray
import random
import torchvision.transforms as transforms

import re
import nltk
from nltk.corpus import stopwords

!pip install transformers

from transformers import BertConfig, BertTokenizer
from transformers import BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.1 MB/s eta 0:00:00


## Device Configuration

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Create Data Loader Class

In [5]:
def save_function(function, directory):
    code_string = marshal.dumps(function.__code__)
    with open(directory, 'wb') as file:
        pickle.dump(code_string, file)
    file.close()

In [6]:
def ToNumpy():
    return pd.DataFrame.to_numpy
def ToTensor():
    return transforms.ToTensor

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, path_label, path_sentiment, path_dialoges, drive_id, mode,transform, image_size, batch_size=None):
    'Initialization'
    self.path_label = path_label
    self.path_sentiment = path_sentiment
    self.path_dialoges = path_dialoges
    self.drive_id = drive_id
    self.mode = mode
    self.image_size = image_size
    self.batch_size = batch_size
    self.ds = self.load_images()
    self.labels = np. asarray(self.read_txt(path_label)[:-1])
    self.indexes = np. asarray(self.conversations_index()[:-1])
    self.dialoges = np. asarray(self.read_txt(path_dialoges)[:-1])
    self.transform = transform
    self.convertType()

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.labels)

  def __getitem__(self, index):
    'Generates one sample of data'
    image_index = self.get_image(index)
    dialoge_index = self.dialoges[index]
    label_index = self.labels[index]
    return image_index, dialoge_index, label_index
  
  def read_txt(self, path_text):
    'Reads text and turn them into a list'
    my_file = open(path_text, "r")
    data = my_file.read()
    data_into_list = data.split("\n")
    return data_into_list

  def load_images(self):
    'download images'
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_id = self.drive_id
    file_path = './DLproject'
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    path_zip = '/content/' + self.mode + '.zip'
    !gdown --id "$file_id"
    !unzip "$path_zip" -d '/content/DLproject/'

    img_height, img_width = self.image_size
    dir_files = '/content/DLproject'
    ds = tf.keras.utils.image_dataset_from_directory(
        dir_files, image_size=(img_height, img_width), batch_size=self.batch_size
    )
    return ds

  def get_image(self, index):
    file_name = '/content/DLproject/'+ self.mode + '//'+ str(index) + '.jpg'
    image = cv2.imread(file_name)
    image= np.asarray(image)
    if(self.transform == transforms.ToTensor):
        trans = transforms.Compose([transforms.ToTensor()])
        image = trans(image)

    return image

  def conversations_index(self):
    'map dialoge index to an image'
    my_file = open(self.path_sentiment, "r")
    data = my_file.read()
    conversations = data.split("\n")
    indexes = []
    for i in range(len(conversations)):
      conversation_i = conversations[i]
      conversation_i = conversation_i[1: -1]
      conversation_i = conversation_i.split(",")
      dialoges_per_conversation = len(conversation_i)
      for j in range(dialoges_per_conversation):
        indexes.append(i)
    return indexes

  def convertType(self):
      if(self.transform == transforms.ToTensor):
          self.dialoges = tf.convert_to_tensor(self.dialoges)
          self.indexes = tf.convert_to_tensor(self.indexes)
          self.labels = tf.convert_to_tensor(self.indexes)

## Import Dataset and Create Dataset Object

In [7]:
classes = ['Neutral', 'Negative', 'Positive']

In [8]:
id = '1EYjMT2_0EpmZQSbWtsaQOMkP_9Zxt8Ll'
data_loader_train = Dataset('sentiment_train.txt', 'image_index_train.txt', 'english_train.txt', id, 'train_ende',transform=ToNumpy(), image_size=(554,1280), batch_size=64)
clear_output()

In [9]:
# id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
# data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToTensor(), image_size=(554,1280), batch_size=64)
# clear_output()

In [10]:
id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

In [11]:
id = '1SlsoJ0X4C-IK6CQQ7L93Od4MrK6TRPv9'
data_loader_val = Dataset('sentiment_dev.txt', 'image_index_dev.txt', 'english_dev.txt', id, 'dev',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

## Import Models

In [12]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids,
            return_dict=False)
        return self.classifier(self.dropout(pooled_output))

class Dataset2(torch.utils.data.Dataset):
    def __init__(self, tokenizer, dialoge_texts, targets=None, labels=None, max_len=None):
        self.dialoge_texts = dialoge_texts
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)
        self.label_map = {label: i for i, label in enumerate(labels)} if isinstance(labels, list) else {}
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.dialoge_texts)

    def __getitem__(self, item):
        dialoge_text = self.dialoge_texts[item]

        if self.has_target:
            target = self.label_map.get(self.targets[item], self.targets[item])
        encoding = self.tokenizer.encode_plus(
            dialoge_text, add_special_tokens=True, truncation=True, max_length=self.max_len, return_token_type_ids=True,
            padding='max_length', return_attention_mask=True, return_tensors='pt')
        
        inputs = { 'dialoge_text': dialoge_text, 'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(), 'token_type_ids': encoding['token_type_ids'].flatten(),}

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
            
        return inputs


image_model = torch.load('/content/drive/MyDrive/best_image_model').to(device)

text_model = torch.load('/content/drive/MyDrive/best_text_model').to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

image_model.fc = nn.Identity()
text_model.classifier = nn.Identity()

# Prepare Text Input

In [13]:
def preprocess_text(text: str, remove_stopwords: bool):
    text = re.sub(r"http\S+", "", text)
    text = re.sub("[^A-Za-z]+", " ", text)
    if remove_stopwords:
        tokens = nltk.word_tokenize(text)
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        text = " ".join(tokens)
    text = text.lower().strip()
    return text

def get_clean_text(documents):
    texts = []
    for i in tqdm(range(len(documents))):
        text = preprocess_text(documents[i], remove_stopwords=False)
        texts.append(text)
    return texts

train_texts = get_clean_text(data_loader_train.dialoges)
val_texts = get_clean_text(data_loader_val.dialoges)
test_texts = get_clean_text(data_loader_test.dialoges)

max_len_1 = max(len(text.split(' ')) for text in train_texts)
max_len_2 = max(len(text.split(' ')) for text in val_texts)
max_len_3 = max(len(text.split(' ')) for text in test_texts)
max_len = max(max_len_1, max_len_2, max_len_3)

  0%|          | 0/20240 [00:00<?, ?it/s]

  0%|          | 0/5063 [00:00<?, ?it/s]

  0%|          | 0/5067 [00:00<?, ?it/s]

In [14]:
def predict_text(model, targets, tokenizer, max_len=max_len, batch_size=32):

    dataset_p = Dataset2(tokenizer, targets, None, None, max_len)
    data_loader = torch.utils.data.DataLoader(dataset_p, batch_size=batch_size)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in data_loader:
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
            
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [15]:
%%time
y_train, y_train_probs = predict_text(text_model, train_texts, tokenizer, max_len=max_len)
y_val, y_val_probs = predict_text(text_model, val_texts, tokenizer, max_len=max_len)
y_test, y_test_probs = predict_text(text_model, test_texts, tokenizer, max_len=max_len)

CPU times: user 34.3 s, sys: 258 ms, total: 34.5 s
Wall time: 38.3 s


In [16]:
text_x_train = y_train_probs
text_x_val = y_val_probs
text_x_test = y_test_probs

In [17]:
%%time
np.save('text_x_train', text_x_train)
np.save('text_x_val', text_x_val)
np.save('text_x_test', text_x_test)

CPU times: user 3.24 ms, sys: 90.8 ms, total: 94 ms
Wall time: 97.5 ms


In [ ]:
os.kill(os.getpid(), 9)

# Initial Setups

## Set MSCTD Configurations

In [12]:
!pip install --upgrade --no-cache-dir gdown
!pip3 install --upgrade gdown
!pip install -U --no-cache-dir gdown --pre
!pip install --upgrade --no-cache-dir gdown
# 
!git clone https://github.com/XL2248/MSCTD

!cp MSCTD/MSCTD_data/ende/english_*.txt .
!cp MSCTD/MSCTD_data/ende/image_index_*.txt .
!cp MSCTD/MSCTD_data/ende/sentiment*.txt .

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'MSCTD' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install einops
!pip install pytorch_lightning
!pip install wandb
!pip install ml_collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries

In [14]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
from tqdm.notebook import tqdm
from time import localtime
import gc

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from IPython.display import clear_output 
from PIL import Image, ImageOps, ImageEnhance

import pytorch_lightning as pl
import torchvision.datasets as dset
import torchvision.transforms as T
from matplotlib import image as mpimg
from PIL import Image
from PIL import Image, ImageOps, ImageEnhance
from einops import parse_shape, rearrange
from opt_einsum import contract
import math
import functools
import pickle, marshal
from einops import rearrange
from numpy import asarray
import random
import torchvision.transforms as transforms

import re
import nltk
from nltk.corpus import stopwords

!pip install transformers

from transformers import BertConfig, BertTokenizer
from transformers import BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Device Configuration

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Create Data Loader Class

In [16]:
def save_function(function, directory):
    code_string = marshal.dumps(function.__code__)
    with open(directory, 'wb') as file:
        pickle.dump(code_string, file)
    file.close()

In [17]:
def ToNumpy():
    return pd.DataFrame.to_numpy
def ToTensor():
    return transforms.ToTensor

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, path_label, path_sentiment, path_dialoges, drive_id, mode,transform, image_size, batch_size=None):
    'Initialization'
    self.path_label = path_label
    self.path_sentiment = path_sentiment
    self.path_dialoges = path_dialoges
    self.drive_id = drive_id
    self.mode = mode
    self.image_size = image_size
    self.batch_size = batch_size
    self.ds = self.load_images()
    self.labels = np. asarray(self.read_txt(path_label)[:-1])
    self.indexes = np. asarray(self.conversations_index()[:-1])
    self.dialoges = np. asarray(self.read_txt(path_dialoges)[:-1])
    self.transform = transform
    self.convertType()

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.labels)

  def __getitem__(self, index):
    'Generates one sample of data'
    image_index = self.get_image(index)
    dialoge_index = self.dialoges[index]
    label_index = self.labels[index]
    return image_index, dialoge_index, label_index
  
  def read_txt(self, path_text):
    'Reads text and turn them into a list'
    my_file = open(path_text, "r")
    data = my_file.read()
    data_into_list = data.split("\n")
    return data_into_list

  def load_images(self):
    'download images'
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_id = self.drive_id
    file_path = './DLproject'
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    path_zip = '/content/' + self.mode + '.zip'
    !gdown --id "$file_id"
    !unzip "$path_zip" -d '/content/DLproject/'

    img_height, img_width = self.image_size
    dir_files = '/content/DLproject'
    ds = tf.keras.utils.image_dataset_from_directory(
        dir_files, image_size=(img_height, img_width), batch_size=self.batch_size
    )
    return ds

  def get_image(self, index):
    file_name = '/content/DLproject/'+ self.mode + '//'+ str(index) + '.jpg'
    image = cv2.imread(file_name)
    image= np.asarray(image)
    if(self.transform == transforms.ToTensor):
        trans = transforms.Compose([transforms.ToTensor()])
        image = trans(image)

    return image

  def conversations_index(self):
    'map dialoge index to an image'
    my_file = open(self.path_sentiment, "r")
    data = my_file.read()
    conversations = data.split("\n")
    indexes = []
    for i in range(len(conversations)):
      conversation_i = conversations[i]
      conversation_i = conversation_i[1: -1]
      conversation_i = conversation_i.split(",")
      dialoges_per_conversation = len(conversation_i)
      for j in range(dialoges_per_conversation):
        indexes.append(i)
    return indexes

  def convertType(self):
      if(self.transform == transforms.ToTensor):
          self.dialoges = tf.convert_to_tensor(self.dialoges)
          self.indexes = tf.convert_to_tensor(self.indexes)
          self.labels = tf.convert_to_tensor(self.indexes)

## Import Dataset and Create Dataset Object

In [18]:
classes = ['Neutral', 'Negative', 'Positive']

In [19]:
id = '1EYjMT2_0EpmZQSbWtsaQOMkP_9Zxt8Ll'
data_loader_train = Dataset('sentiment_train.txt', 'image_index_train.txt', 'english_train.txt', id, 'train_ende',transform=ToNumpy(), image_size=(554,1280), batch_size=64)
clear_output()

In [20]:
# id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
# data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToTensor(), image_size=(554,1280), batch_size=64)
# clear_output()

In [21]:
id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

In [22]:
id = '1SlsoJ0X4C-IK6CQQ7L93Od4MrK6TRPv9'
data_loader_val = Dataset('sentiment_dev.txt', 'image_index_dev.txt', 'english_dev.txt', id, 'dev',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

## Import Models

In [23]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids,
            return_dict=False)
        return self.classifier(self.dropout(pooled_output))

class Dataset2(torch.utils.data.Dataset):
    def __init__(self, tokenizer, dialoge_texts, targets=None, labels=None, max_len=None):
        self.dialoge_texts = dialoge_texts
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)
        self.label_map = {label: i for i, label in enumerate(labels)} if isinstance(labels, list) else {}
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.dialoge_texts)

    def __getitem__(self, item):
        dialoge_text = self.dialoge_texts[item]

        if self.has_target:
            target = self.label_map.get(self.targets[item], self.targets[item])
        encoding = self.tokenizer.encode_plus(
            dialoge_text, add_special_tokens=True, truncation=True, max_length=self.max_len, return_token_type_ids=True,
            padding='max_length', return_attention_mask=True, return_tensors='pt')
        
        inputs = { 'dialoge_text': dialoge_text, 'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(), 'token_type_ids': encoding['token_type_ids'].flatten(),}

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
            
        return inputs


image_model = torch.load('/content/drive/MyDrive/best_image_model').to(device)

text_model = torch.load('/content/drive/MyDrive/best_text_model').to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

image_model.fc = nn.Identity()
text_model.classifier = nn.Identity()

# Prepare Image Input

In [24]:
def get_data(data_loader: Dataset, frame_size = (128, 128), skip=1):
    labels = []
    frames = []
    N = data_loader.__len__()
    for i in tqdm(range(0, N, skip)):
        frame, _, label = data_loader.__getitem__(i)
        frames.append(cv2.resize(frame, dsize = frame_size).astype('float32'))
        labels.append(label)
    return frames , labels

In [62]:
train_frames , train_labels = get_data(data_loader_train, skip=4)
val_frames , val_labels = get_data(data_loader_val, skip=4)
test_frames , test_labels = get_data(data_loader_test, skip=4)

  0%|          | 0/1267 [00:00<?, ?it/s]

In [63]:
def getTime():
  loc_time = localtime()
  hour, min, sec = loc_time.tm_hour, loc_time.tm_min, loc_time.tm_sec
  return str(hour) + ":" + str(min) + ":" + str(sec)


def test_model(net, criterion, dataloader):
    with torch.no_grad():
        pred = torch.tensor([]) #.to(device)
        net_cpu = net.to('cpu')
        net.eval()
        for i, (x, y) in tqdm(enumerate(dataloader)):
            # x = x.to(device)
            # y = y.to(device)
            p = net(x)
            # y_hat = torch.cat((y_hat, p), 0)
            # predictions = p.argmax(-1)
            pred = torch.cat((pred, p), 0)

    return pred


def toLabel(x):
    if x == 'Positive' or x == '2':
        return 2
    elif x == 'Negative' or x == '1':
        return 1
    else:
        return 0

train_frames_r = np.zeros((len(train_frames), 3, 128 , 128))
val_frames_r = np.zeros((len(val_frames), 3, 128 , 128))
test_frames_r = np.zeros((len(test_frames), 3, 128, 128))

if train_frames[0].shape[2] == 3:
    for i in range(len(train_frames)):
        train_frames_r[i] = train_frames[i].T

if val_frames[0].shape[2] == 3:
    for i in range(len(val_frames)):
        val_frames_r[i] = val_frames[i].T

if test_frames[0].shape[2] == 3:
    for i in range(len(test_frames)):
        test_frames_r[i] = test_frames[i].T


y_train = [toLabel(i) for i in train_labels]
y_val = [toLabel(i) for i in val_labels]
y_test = [toLabel(i) for i in test_labels]

x_train = torch.tensor(np.array(train_frames_r), dtype=torch.float32)
y_train = torch.tensor(y_train).type(torch.LongTensor)
x_val = torch.tensor(np.array(val_frames_r), dtype=torch.float32)
y_val = torch.tensor(y_val).type(torch.LongTensor)
x_test = torch.tensor(np.array(test_frames_r), dtype=torch.float32)
y_test = torch.tensor(y_test).type(torch.LongTensor)


train_set = list(map(lambda x, y: (x, y), x_train, y_train))
val_set = list(map(lambda x, y: (x, y), x_val, y_val))
test_set = list(map(lambda x, y: (x, y), x_test, y_test))

batch_size = 1

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2 )
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [64]:
criterion = nn.CrossEntropyLoss()

train_pred = test_model(image_model, criterion, train_loader)
val_pred = test_model(image_model, criterion, val_loader)
test_pred = test_model(image_model, criterion, test_loader)

0it [00:00, ?it/s]

In [66]:
image_x_train = train_pred
image_x_val = val_pred
image_x_test = test_pred

In [67]:
%%time
np.save('image_x_train', image_x_train)
np.save('image_x_val', image_x_val)
np.save('image_x_test', image_x_test)

CPU times: user 0 ns, sys: 13.6 ms, total: 13.6 ms
Wall time: 22.3 ms


In [68]:
drive_path = F"/content/drive/MyDrive/image_x_test" 
torch.save(image_x_test, drive_path)

In [ ]:
os.kill(os.getpid(), 9)

# Initial Setups

## Set MSCTD Configurations

In [22]:
!pip install --upgrade --no-cache-dir gdown
!pip3 install --upgrade gdown
!pip install -U --no-cache-dir gdown --pre
!pip install --upgrade --no-cache-dir gdown
# 
!git clone https://github.com/XL2248/MSCTD

!cp MSCTD/MSCTD_data/ende/english_*.txt .
!cp MSCTD/MSCTD_data/ende/image_index_*.txt .
!cp MSCTD/MSCTD_data/ende/sentiment*.txt .

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'MSCTD' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!pip install einops
!pip install pytorch_lightning
!pip install wandb
!pip install ml_collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries

In [24]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
from tqdm.notebook import tqdm
from time import localtime
import gc

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from IPython.display import clear_output 
from PIL import Image, ImageOps, ImageEnhance

from numpy import asarray
import random
import torchvision.transforms as transforms

import re
import nltk
from nltk.corpus import stopwords

!pip install transformers

from transformers import BertConfig, BertTokenizer
from transformers import BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Device Configuration

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Create Data Loader Class

In [34]:
def save_function(function, directory):
    code_string = marshal.dumps(function.__code__)
    with open(directory, 'wb') as file:
        pickle.dump(code_string, file)
    file.close()

In [35]:
def ToNumpy():
    return pd.DataFrame.to_numpy
def ToTensor():
    return transforms.ToTensor

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, path_label, path_sentiment, path_dialoges, drive_id, mode,transform, image_size, batch_size=None):
    'Initialization'
    self.path_label = path_label
    self.path_sentiment = path_sentiment
    self.path_dialoges = path_dialoges
    self.drive_id = drive_id
    self.mode = mode
    self.image_size = image_size
    self.batch_size = batch_size
    self.ds = self.load_images()
    self.labels = np. asarray(self.read_txt(path_label)[:-1])
    self.indexes = np. asarray(self.conversations_index()[:-1])
    self.dialoges = np. asarray(self.read_txt(path_dialoges)[:-1])
    self.transform = transform
    self.convertType()

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.labels)

  def __getitem__(self, index):
    'Generates one sample of data'
    image_index = self.get_image(index)
    dialoge_index = self.dialoges[index]
    label_index = self.labels[index]
    return image_index, dialoge_index, label_index
  
  def read_txt(self, path_text):
    'Reads text and turn them into a list'
    my_file = open(path_text, "r")
    data = my_file.read()
    data_into_list = data.split("\n")
    return data_into_list

  def load_images(self):
    'download images'
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_id = self.drive_id
    file_path = './DLproject'
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    path_zip = '/content/' + self.mode + '.zip'
    !gdown --id "$file_id"
    !unzip "$path_zip" -d '/content/DLproject/'

    img_height, img_width = self.image_size
    dir_files = '/content/DLproject'
    ds = tf.keras.utils.image_dataset_from_directory(
        dir_files, image_size=(img_height, img_width), batch_size=self.batch_size
    )
    return ds

  def get_image(self, index):
    file_name = '/content/DLproject/'+ self.mode + '//'+ str(index) + '.jpg'
    image = cv2.imread(file_name)
    image= np.asarray(image)
    if(self.transform == transforms.ToTensor):
        trans = transforms.Compose([transforms.ToTensor()])
        image = trans(image)

    return image

  def conversations_index(self):
    'map dialoge index to an image'
    my_file = open(self.path_sentiment, "r")
    data = my_file.read()
    conversations = data.split("\n")
    indexes = []
    for i in range(len(conversations)):
      conversation_i = conversations[i]
      conversation_i = conversation_i[1: -1]
      conversation_i = conversation_i.split(",")
      dialoges_per_conversation = len(conversation_i)
      for j in range(dialoges_per_conversation):
        indexes.append(i)
    return indexes

  def convertType(self):
      if(self.transform == transforms.ToTensor):
          self.dialoges = tf.convert_to_tensor(self.dialoges)
          self.indexes = tf.convert_to_tensor(self.indexes)
          self.labels = tf.convert_to_tensor(self.indexes)

## Import Dataset and Create Dataset Object

In [28]:
classes = ['Neutral', 'Negative', 'Positive']

In [29]:
id = '1EYjMT2_0EpmZQSbWtsaQOMkP_9Zxt8Ll'
data_loader_train = Dataset('sentiment_train.txt', 'image_index_train.txt', 'english_train.txt', id, 'train_ende',transform=ToNumpy(), image_size=(554,1280), batch_size=64)
clear_output()

In [30]:
# id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
# data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToTensor(), image_size=(554,1280), batch_size=64)
# clear_output()

In [31]:
id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

In [32]:
id = '1SlsoJ0X4C-IK6CQQ7L93Od4MrK6TRPv9'
data_loader_val = Dataset('sentiment_dev.txt', 'image_index_dev.txt', 'english_dev.txt', id, 'dev',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

# Concat input vectors

## Import libraries & function

In [2]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from time import localtime

In [3]:
def train_epoch(net, criterion, optimizer, dataloader):
  epoch_loss = 0
  epoch_true = 0
  epoch_all = 0
  train_loss = 0
  train_accuracy = 0
  i = 0

  net.train()
  for i, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)
    p = net(x)

    loss = criterion(p, y)
    epoch_loss += float(loss)

    predictions = p.argmax(-1)
    epoch_all += len(predictions)
    epoch_true += (predictions == y).sum()
    
    train_loss = epoch_loss / (i + 1)
    train_accuracy = epoch_true / epoch_all

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

  return train_loss, train_accuracy


def evaluate_epoch(net, criterion, dataloader):
  epoch_loss = 0
  epoch_true = 0
  epoch_all = 0
  eval_loss = 0
  eval_accuracy = 0
  i = 0

  net.eval()
  for i, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)
    p = net(x)

    loss = criterion(p, y)
    epoch_loss += float(loss)

    predictions = p.argmax(-1)
    epoch_all += len(predictions)
    epoch_true += (predictions == y).sum()

    eval_loss = epoch_loss / (i + 1)
    eval_accuracy = epoch_true / epoch_all

  return eval_loss, eval_accuracy


def getTime():
  loc_time = localtime()
  hour, min, sec = loc_time.tm_hour, loc_time.tm_min, loc_time.tm_sec
  return str(hour) + ":" + str(min) + ":" + str(sec)


def train_model(net, n_epochs, criterion, optimizer, train_loader, test_loader=None, best_model_name=''):
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []
    max_test_acc = 0
    if best_model_name == '':
        best_model_name = 'model.pth'

    for e in range(n_epochs):
        train_loss, train_accuracy = train_epoch(net, criterion, optimizer, train_loader)
        if test_loader is not None:
            test_loss, test_accuracy = evaluate_epoch(net, criterion, test_loader)

        train_losses.append(train_loss)
        train_accuracies.append(float(train_accuracy))
        if test_loader is not None:
            test_losses.append(test_loss)
            test_accuracies.append(float(test_accuracy))
            if float(test_accuracy) > max_test_acc:
                max_test_acc = float(test_accuracy)
                torch.save(net, best_model_name)

        print(f'[Epoch {e+1}]\t [{getTime()}]\t', end="")
        print(f'Train Loss: {train_loss:.4f}\tTrain Accuracy: {train_accuracy:.2f}\t', end="")
        if test_loader is not None:
            print(f'Test Loss: {test_loss:.4f}\tTest Accuracy: {test_accuracy:.2f}')
        else:
            print()
    
    return train_losses, train_accuracies, test_losses, test_accuracies


def test_model(net, criterion, dataloader):
    epoch_loss = 0
    epoch_true = 0
    epoch_all = 0
    eval_loss = 0
    eval_accuracy = 0
    i = 0
    with torch.no_grad():
        pred = torch.tensor([]) #.to(device)

        net_cpu = net.to('cpu')
        net.eval()
        for i, (x, y) in enumerate(dataloader):
            # x = x.to(device)
            # y = y.to(device)
            
            p = net(x)
            # y_hat = torch.cat((y_hat, p), 0)
            
            loss = criterion(p, y)
            epoch_loss += float(loss)

            predictions = p.argmax(-1)
            epoch_all += len(predictions)
            epoch_true += (predictions == y).sum()

            eval_loss = epoch_loss / (i + 1)
            eval_accuracy = epoch_true / epoch_all

            pred = torch.cat((pred, predictions), 0)

    return pred, eval_loss, eval_accuracy

## Load data

In [4]:
image_x_train = np.load('image_x_train.npy')
image_x_val = np.load('image_x_val.npy')
image_x_test = np.load('image_x_test.npy')

In [9]:
text_x_train = np.load('text_x_train.npy')
text_x_val = np.load('text_x_val.npy')
text_x_test = np.load('text_x_test.npy')

In [10]:
text_x_train = text_x_train[::4]
text_x_val = text_x_val[::4]
text_x_test = text_x_test[::4]

In [11]:
print(image_x_train.shape, text_x_train.shape)
print(image_x_val.shape, text_x_val.shape)
print(image_x_test.shape, text_x_test.shape)

(5060, 2048) (5060, 768)
(1266, 2048) (1266, 768)
(1267, 2048) (1267, 768)


In [15]:
X_train = np.concatenate((image_x_train, text_x_train), axis=1)
X_val = np.concatenate((image_x_val, text_x_val), axis=1)
X_test = np.concatenate((image_x_test, text_x_test), axis=1)

In [17]:
print(X_train.shape, X_val.shape, X_test.shape)

(5060, 2816) (1266, 2816) (1267, 2816)


In [37]:
def toLabel(x):
    if x == 'Positive' or x == '2':
        return 2
    elif x == 'Negative' or x == '1':
        return 1
    else:
        return 0


train_labels = data_loader_train.labels[::4]
val_labels = data_loader_val.labels[::4]
test_labels = data_loader_test.labels[::4]

y_train = [toLabel(i) for i in train_labels]
y_val = [toLabel(i) for i in val_labels]
y_test = [toLabel(i) for i in test_labels]

x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train).type(torch.LongTensor)
x_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val).type(torch.LongTensor)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test).type(torch.LongTensor)

train_set = list(map(lambda x, y: (x, y), x_train, y_train))
val_set = list(map(lambda x, y: (x, y), x_val, y_val))
test_set = list(map(lambda x, y: (x, y), x_test, y_test))

batch_size = 128

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [38]:
class CombineModel(nn.Module):
    def __init__(self, input_dim):
        super(CombineModel, self).__init__()

        self.input_dim = input_dim
        self.hidden_dims = [512, 64, 8]
        self.output_dim = 3
        
        self.fc = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
            nn.ReLU(),
            nn.Linear(self.hidden_dims[2], self.output_dim),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, x):
        return self.fc(x)

In [39]:
input_dim = X_train.shape[1]

net = CombineModel(input_dim=input_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)
# optimizer = torch.optim.SGD(net_tfidf.parameters(), lr=0.001, momentum=0.9)

In [40]:
train_loss, train_acc, _, _= train_model(net, 15, criterion, optimizer, train_loader, val_loader, best_model_name='classify_model.pth')

[Epoch 1]	 [3:45:2]	Train Loss: 1.0935	Train Accuracy: 0.37	Test Loss: 1.0959	Test Accuracy: 0.36
[Epoch 2]	 [3:45:3]	Train Loss: 1.0880	Train Accuracy: 0.39	Test Loss: 1.0962	Test Accuracy: 0.36
[Epoch 3]	 [3:45:4]	Train Loss: 1.0873	Train Accuracy: 0.39	Test Loss: 1.0981	Test Accuracy: 0.36
[Epoch 4]	 [3:45:4]	Train Loss: 1.0811	Train Accuracy: 0.40	Test Loss: 1.1023	Test Accuracy: 0.35
[Epoch 5]	 [3:45:5]	Train Loss: 1.0760	Train Accuracy: 0.42	Test Loss: 1.1072	Test Accuracy: 0.35
[Epoch 6]	 [3:45:5]	Train Loss: 1.0715	Train Accuracy: 0.42	Test Loss: 1.1224	Test Accuracy: 0.35
[Epoch 7]	 [3:45:6]	Train Loss: 1.0657	Train Accuracy: 0.44	Test Loss: 1.1211	Test Accuracy: 0.35
[Epoch 8]	 [3:45:7]	Train Loss: 1.0567	Train Accuracy: 0.44	Test Loss: 1.1186	Test Accuracy: 0.36
[Epoch 9]	 [3:45:7]	Train Loss: 1.0545	Train Accuracy: 0.45	Test Loss: 1.1244	Test Accuracy: 0.36
[Epoch 10]	 [3:45:8]	Train Loss: 1.0497	Train Accuracy: 0.46	Test Loss: 1.1418	Test Accuracy: 0.35
[Epoch 11]	 [3:45:9

In [41]:
classify_model = torch.load('classify_model.pth')

y_pred, loss, acc = test_model(classify_model, criterion, test_loader)

print(f'accuracy: {acc}')

accuracy: 0.4254143536090851


# Predict for Sample

In [ ]:
def string_label(x):
    if x == 2:
        return 'Positive' 
    elif x == 1:
        return 'Negative' 
    else:
        return 'Neutral'


def predict_for_sample(image, text):
    image = torch.tensor([image.T], dtype=torch.float32).to(device)
    with torch.no_grad():
        pred_image = image_model(image)
    # print(f'predict from image: {pred_image.shape}')

    text = np.array(np.array([text]))
    y_pred_text, pred_text = predict_text(text_model, text, tokenizer, max_len=max_len)
    # print(f'\npredict from text: {pred_text[0].shape}')
    pred_text = torch.tensor(pred_text, dtype=torch.float32).to(device)
    return pred_image[0], pred_text[0]


def create_input_vector_for_sample(image, text):
    y_image, y_text = predict_for_sample(image, text)
    # print(y_image.shape, y_text.shape)
    y = torch.cat((y_image, y_text), dim=0)
    return y

In [ ]:
i = 150
image, _, label = data_loader_test.__getitem__(i)
dialoge = test_texts[i]
print(f'{dialoge}, {label}\n')

y_image, y_text = predict_for_sample(image, dialoge)

print(f'image model label: {y_image.shape},\ntext model label {y_text.shape}')

not really the pun type jahrling, 1

image model label: torch.Size([2048]),
text model label torch.Size([768])
